# First Test Basic VaR Models 

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np

# Import functions
import basic_var 
import backtesting as BT
import plots
import expected_shortfall as ES
import data_download as dd

### Data

In [15]:
'''# Download data
sp500_data = yf.download("^GSPC", start="2000-01-01", end="2024-01-01")
sp500_data["Log Returns"] = np.log(sp500_data["Close"] / sp500_data["Close"].shift(1))
returns = sp500_data["Log Returns"].dropna()'''

'# Download data\nsp500_data = yf.download("^GSPC", start="2000-01-01", end="2024-01-01")\nsp500_data["Log Returns"] = np.log(sp500_data["Close"] / sp500_data["Close"].shift(1))\nreturns = sp500_data["Log Returns"].dropna()'

# DATA USANDO LE NOSTRE FUNZIONI


In [ ]:
# Define tickers and number of shares
tickers = ["MSFT", "NVDA", "AAPL"]
shares = pd.Series({"MSFT": 3, "NVDA": 3, "AAPL": 3})

# Download price data
prices = dd.get_raw_prices(tickers, start="2015-01-01")
prices = dd.convert_to_base(prices, base='EUR', show_currency_detection=False)

# Compute portfolio value each day
portfolio_value = (prices * shares).sum(axis=1)

# Compute simple daily returns
returns = portfolio_value.pct_change().dropna()

# Return as DataFrame (same format as SP500 example)
portfolio_returns_df = returns.to_frame(name="Returns")

returns = portfolio_returns_df["Returns"].dropna()
returns.head()

Date
2020-01-03   -0.007318
2020-01-06    0.005169
2020-01-07   -0.010377
2020-01-08    0.019549
2020-01-09    0.019013
Name: Returns, dtype: float64

In [6]:
# Set parameters
confidence_level = 0.99 # <----- Can choose 0.95 etc

### Historical VaR

In [ ]:
# Apply Historical VaR model
historical_output, var = basic_var.var_historical(returns, confidence_level)

print(f"Historical VaR estimate (abs): {var:.2f}%")

# Compute ES for the whole period
historical_output, es_estimate = ES.es_historical(historical_output)

print(f"Historical ES estimate (abs): {100 * es_estimate:.2f}%")


# Plot interactive VaR
fig_var = plots.plot_backtest(historical_output, interactive=False)



Historical VaR estimate (abs): 5.35%
Historical ES estimate (abs): 7.05%


In [ ]:
historical_output.head()

,Returns,VaR,VaR Violation,ES
Date,,,,
2020-01-03,-0.011688,0.047872,False,0.066165
2020-01-06,0.004317,0.047872,False,0.066165
2020-01-07,-0.007183,0.047872,False,0.066165
2020-01-08,0.015611,0.047872,False,0.066165
2020-01-09,0.015219,0.047872,False,0.066165


### Parametric VaR

In [ ]:
# Apply Parametric VaR model (choose distribution: "normal", "t", or "ged")
param_output, var = basic_var.var_parametric(returns, confidence_level, distribution="t")

print(f"Parametric VaR estimate (abs): {var:.2f}%")

# Compute ES for the whole period
param_output, es_estimate = ES.es_parametric(param_output, returns, confidence_level, distribution="t")

print(f"Parametric ES estimate (abs): {100 * es_estimate:.2f}%")


# Plot interactive VaR and ES for a subset
fig_es = plots.plot_backtest(param_output, subset=("2019-11-01", "2020-11-30"), interactive=False)


Parametric VaR estimate (abs): 5.06%
Parametric ES estimate (abs): 7.08%


'\n# Plot interactive VaR and ES for a subset\nfig_es = plots.plot_backtest(param_output, subset=("2019-11-01", "2020-11-30"), interactive=False)\n'

### Test Wealth Input

In [ ]:
wealth = portfolio_value.iloc[-1]
# Apply Parametric VaR model (choose distribution: "normal", "t", or "ged")
param_output_wealth, var_wealth = basic_var.var_parametric(returns, confidence_level, distribution="t", wealth=wealth)

print(f"Parametric VaR estimate $ (abs): {var_wealth:.2f}")

param_output_wealth.head()

Parametric VaR estimate $ (abs): 114.37


,Returns,VaR,VaR Violation,VaR_monetary
Date,,,,
2020-01-03,-0.011688,0.050566,False,114.367249
2020-01-06,0.004317,0.050566,False,114.367249
2020-01-07,-0.007183,0.050566,False,114.367249
2020-01-08,0.015611,0.050566,False,114.367249
2020-01-09,0.015219,0.050566,False,114.367249


In [ ]:
# Compute ES for the whole period
param_output_es_wealth, es_estimate_wealth = ES.es_parametric(param_output, returns, confidence_level, distribution="t", wealth=wealth)

print(f"Parametric ES estimate $ (abs): {es_estimate_wealth:.2f}")

param_output_es_wealth.head()   

Parametric ES estimate $ (abs): 160.02


,Returns,VaR,VaR Violation,ES,ES_monetary
Date,,,,,
2020-01-03,-0.011688,0.050566,False,0.070753,160.023055
2020-01-06,0.004317,0.050566,False,0.070753,160.023055
2020-01-07,-0.007183,0.050566,False,0.070753,160.023055
2020-01-08,0.015611,0.050566,False,0.070753,160.023055
2020-01-09,0.015219,0.050566,False,0.070753,160.023055


### Backtesting

In [ ]:
total_violations, violation_rate = BT.count_violations(param_output)

print(f"Total VaR Violations: {total_violations}")
print(f"Violation Rate: {violation_rate:.2f}%")


Total VaR Violations: 11
Violation Rate: 0.82%


In [ ]:
kupiec_results = BT.kupiec_test(
    total_violations=total_violations,
    total_days=len(param_output),
    confidence_level=confidence_level
)

print("\nKupiec Test Results:")
for key, value in kupiec_results.items():
    print(f"{key}: {value}")


Kupiec Test Results:
LR_uc: 0.4806608105848653
p_value: 0.4881231489719102
reject_null: False


In [ ]:
christoffersen_results = BT.christoffersen_test(param_output)

print("\nChristoffersen Test Results:")
for key, value in christoffersen_results.items():
    print(f"{key}: {value}")



Christoffersen Test Results:
LR_c: 3.1521174432905212
p_value: 0.07582850522567786
reject_null: False


In [ ]:
joint_results = BT.joint_lr_test(
    LR_uc=kupiec_results["LR_uc"],
    LR_c=christoffersen_results["LR_c"]
)

print("\nJoint Test Results:")
for key, value in joint_results.items():
    print(f"{key}: {value}")


Joint Test Results:
LR_total: 3.6327782538753866
p_value: 0.16261186290446072
reject_null: False


### Appendix (DOUBLE CHECK)

#### 1. Historical Expected Shortfall (Tail Mean)  
Historical ES is the average of losses that exceed the historical VaR threshold.

$$
\text{ES} = -\mathbb{E}[r_t \mid r_t < -\text{VaR}_\alpha]
$$

#### 2. Parametric Expected Shortfall (Normal or Student-t)  
For the Normal distribution, the ES has a closed-form solution:

$$
\text{ES}_{\text{normal}} = \sigma \cdot \frac{\phi(z_\alpha)}{1 - \alpha}
$$

For the Student-t distribution with $\nu$ degrees of freedom:

$$
\text{ES}_{t} = \sigma \cdot \frac{f_{t_\nu}(t_\alpha)}{1 - \alpha} \cdot \frac{\nu + t_\alpha^2}{\nu - 1}
$$

Where:  
- $z_\alpha$ is the quantile from the standard Normal distribution  
- $t_\alpha$ is the quantile from the Student-t distribution  
- $\phi$ is the PDF of the standard Normal  
- $f_{t_\nu}$ is the PDF of the t-distribution  
- $\sigma$ is the standard deviation or scale parameter
